## Conociendo un poco nuestro DATASET

#### Las limpiezas que se deben realizar son las siguientes:

1- `belongs_to_collection`, `production_companies`: los datos estan anidados
    Se debe buscar la forma de desanidarlos, o bien, buscar la forma de que los datos se puedan consultar sin desanidarlos.

2- `revenue`, `budget`: poseen datos nulos.
    Se deben rellenar estos datos con *0*.

3- `release_date`: posee datos nulos.
    Se deben eliminar estos datos. ¿Se deben eliminar estos registros?

4- `Campos con fecha`: no poseen el formato correcto.
    Se debe transformar cada uno de estos campos al formato *AAAA-mm-dd*.
    Se debe crear una nueva columna `release_year` donde se extraerá el año de la fecha de estreno.

5- `revenue`, `budget`: a partir de estos campos se debe crear una nueva columna `return`, retorno de inversión.
    Esta columna sera el resultado de `revenue`/`budget` y cuando no hay datos disponibles debera retornar *0*.
    
6- `video`, `imbd_id`, `adult`, `original_title`, `poster_path`, `homepage`: no utilizaremos estos campos.
    Se deberan eliminar estos campos.

In [4]:
# importamos las librerias necesarias para limpieza y transformacion:
import pandas as pd
import numpy as np

### Movies

In [5]:
# creamos nuestro dataframe
mv = pd.read_csv('movies_dataset.csv', delimiter=',')
mv.tail(4)

C:\Users\Martin\AppData\Local\Temp\ipykernel_11784\780545667.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  mv = pd.read_csv('movies_dataset.csv', delimiter=',')


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45464,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,1917-10-21,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0
45465,False,NaN,0,[],NaN,461257,tt6980792,en,Queerama,50 years after decriminalisation of homosexual...,...,2017-06-09,0.0,75.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Queerama,False,0.0,0.0


In [6]:
# vemos el tamaño de nuestro dataframe
mv.shape

(45466, 24)

In [7]:
# vemos sus columnas
mv.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

#### 1- `belongs_to_collection`, `production_companies`: los datos estan anidados
    Se debe buscar la forma de desanidarlos, o bien, buscar la forma de que los datos se puedan consultar sin desanidarlos.

In [33]:
mv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [112]:
# Como hariamos para desanidar un solo registro? Tener en cuenta que en algunos registros no hay datos.

x = mv['belongs_to_collection'][0]
y = mv['belongs_to_collection'][12]

In [115]:
import ast
dicc_x = ast.literal_eval(x)
dicc_y = ast.literal_eval(y)
dicc_y.get('id') # le damos la key y devuelve el value

117693